# Calcul matriciel et pièges en numpy

Un des principaux défauts de ```numpy``` (par rapport à des environnement comme Matlab par exemple) est la **différentiation entre les vecteurs et les matrices**... Cet aspect va poser de nombreux problèmes.

Le second défaut réside dans le **dispatch dynamique**: python essaie de faire marcher les calculs, même quand les dimensions des matrices sont incompatibles... Ca fait gagner un peu de temps parfois... Et ça crée des bugs insurmontables d'autres fois.

In [ ]:
import numpy as np

# Produit matriciel

Il ne faut pas confondre le produit terme à terme, entre matrices de mêmes dimensions et le produit matriciel qui est un ensemble de produits scalaires. 
Dans le produit matriciel, le nombre de colonne de la première matrice doit correspondre au nombre de ligne de la seconde/


<img src="./ressources/matrix_mult.png">

$$ {\color {red}c_{{12}}}=\sum _{{r=1}}^{2}a_{{1r}}b_{{r2}}=a_{{11}}b_{{12}}+a_{{12}}b_{{22}}$$

$$ {\color {blue}c_{{33}}}=\sum _{{r=1}}^{2}a_{{3r}}b_{{r3}}=a_{{31}}b_{{13}}+a_{{32}}b_{{23}}$$

In [ ]:
A = np.array([[0., 1], [2, 3], [4, 5], [6, 7]])
B = np.array([[5., 4, 3], [2, 1, 0]])
print("A=",A,"\n B=",B)

# calcul de produit matriciel, par différents moyens
C  = A@B # le plus clair d'après moi
C2 = A.dot(B)

print(C)

In [ ]:
# test sur des matrices incompatibles (matriciel)

A = np.array([[0., 1], [2, 3], [4, 5], [6, 7]])
M = np.array([[0., 1], [2, 3], [4, 5]])

print(A@M)

In [ ]:
# et le produit terme à terme, uniquement entre matrices de même dimension

A = np.array([[0., 1], [2, 3], [4, 5], [6, 7]])
A2 = np.array([[1., 2], [2, 1], [1, 2], [2, 1]])

print(A*A2)

In [ ]:
# test sur des données incompatibles (terme à terme)

C3 = A*B # tournure historiquement très ambigue : 
# heureusement, ne marche plus sur les nouvelles versions de python
# le * est maintenant réservé aux mutliplications terme à terme

## Types de données

In [ ]:
# Les commandes ci-dessous créent des vecteurs

v1=np.random.rand(10)
v2=np.array([1, 4, 18])
v3=np.ones(12)

print(v1.shape)

In [ ]:
# Les commandes ci-dessous créent des matrices

m1=np.random.rand(10,2)
m2=np.array([[1, 4, 18],[2, 4, 6]])
m3=np.ones((12,2))

print(m1.shape)

In [ ]:
# Cas limite

v2 = np.array([1, 4, 18])
# et 
m2 = np.array([[1, 4, 18]])

print("v2 et m2 ne sont pas du même type: ", v2.shape, m2.shape)

In [ ]:
# extraire une ligne ou une colonne => générer un vecteur (et pas une matrice)
# => nous sommes obligé de jongler avec les types de données

m1 = np.random.rand(10,3)
v4 = m1[:,1]   # extraction d'une colonne => vecteur
m4 = m1[:,1:3] # extraction de deux colonnes => matrice
x1 = m1[:,1:2] # extraction d'une seule colonnes, mais en syntaxe matricielle => ???

print(v4.shape, m4.shape)
print(x1.shape) # => matrice !!

### Pourquoi ces différences de types posent problème?

In [ ]:
A     = np.array([[0., 1], [2, 3], [4, 5], [6, 7]])
B_col = np.array([[1.], [2]]) # matrice (=en forme de vecteur colonne)
B_li  = np.array([[1., 2]])   # matrice (=en forme de vecteur ligne)
B_vec = np.array([1., 2])     # vecteur

# print(A@B_li) # KO pour les dimensions => raisonnable, mais attention aux versions de python
print(A@B_col)
print(A@B_vec) # les résultats n'ont pas les mêmes dimensions
print(A.dot(B_col))
print(A.dot(B_vec)) # les résultats n'ont pas les mêmes dimensions

print("\n")

# print(A*B_col) # Erreur => c'est logique
print(A*B_li)    # Catastrophe => ca ne fait pas d'erreur
print(A*B_vec)   # Catastrophe => ca ne fait pas d'erreur

In [ ]:
B_col = np.array([[1.], [2]]) # matrice (=en forme de vecteur colonne)
B_li  = np.array([[1., 2]])   # matrice (=en forme de vecteur ligne)
B_vec = np.array([1., 2])     #

print(B_li@B_col) # le calcul est propre... Et renvoie une matrice
# print(B_li@B_li)  # KO
print(B_vec@B_vec) # produit scalaire
print(B_vec@B_col) # renvoie un vecteur


In [ ]:
# tentons dans l'autre sens
# 
print(B_col @ B_li) # renvoie une matrice (OK)
# print(B_col @ B_vec) # KO : alors qu'on  voudrait intuitivement que ça marche
print(B_vec * B_li) # OK
print(B_vec * B_col) # => WAOUH , carrement n'importe quoi !!

## Dispatch dynamique

Beaucoup de comportements étranges sont liés à cette fonctionnalité. Partons d'un exemple simple et étudions les différentes solutions pratiques:

    # Soit la matrice A:
    A = np.array([[0., 1], [2, 3], [4, 5], [6, 7]])
    # Je souhaite multiplier chaque ligne par le vecteur [1,2]

In [ ]:
# Soit la matrice A:
A = np.array([[0., 1], [2, 3], [4, 5], [6, 7]])

# Je souhaite multiplier chaque ligne par le vecteur [1,2]


In [ ]:
# solution 1: developpeur standard avec des boucles

B = A.copy()
for ligne in B:
    ligne *= [1,2]
print(B)

In [ ]:
# solution 2: raisonnement matriciel, je veux juste multiplier la seconde colonne
B = A.copy()
B[:,1] *= 2
print(B)

In [ ]:
# solution 3: je crée une matrice pour ensuite faire une multiplication terme à terme

M = np.ones((4,2))
M[:,1] *= 2
B = A*M
print(B)

In [ ]:
# solution 4: dispatch dynamique
# ça ne devrait pas marcher... MAIS
#   1. python détecte que le nb de colonne est compatible
#   2. python applique l'opération sur chaque ligne automatiquement
#   => pratique... Mais risqué: il faut connaitre ce truc pour détecter les bugs associés

B = A * [1,2]
print(B)

In [ ]:
# pour faire la même chose sur les colonnes
# à multiplier par [1, 2, 3, 4]

B = A * [[1],[2],[3],[4]] # il faut présenter un vecteur colonne 
print(B)